d #Demand Forecasting

The objective of this notebook is to illustrate how we might generate a large number of fine-grained forecasts at the store-item level in an efficient manner leveraging the distributed computational power of Databricks.  For this exercise, we will make use of an increasingly popular library for demand forecasting, [FBProphet](https://facebook.github.io/prophet/), which we will load into the notebook session associated with a cluster running Databricks 6.0 or higher:

In [2]:
# load fbprophet library
dbutils.library.installPyPI('FBProphet', version='0.5') # find latest version of fbprophet here: https://pypi.org/project/fbprophet/
dbutils.library.installPyPI('holidays','0.9.12') # this line is in response to this issue with fbprophet 0.5: https://github.com/facebook/prophet/issues/1293

dbutils.library.restartPython()

## Examine the Data

For our training dataset, we will make use of 5-years of store-item unit sales data for 50 items across 10 different stores.  This data set is publicly available as part of a past Kaggle competition and can be downloaded [here](https://www.kaggle.com/c/demand-forecasting-kernels-only/data). 

Once downloaded, we can uzip the *train.csv.zip* file and upload the decompressed CSV to */FileStore/tables/demand_forecast/train/* using the file import steps documented [here](https://docs.databricks.com/data/tables.html#create-table-ui). Please note when performing the file import, you don't need to select the *Create Table with UI* or the *Create Table in Notebook* options to complete the import process.

With the dataset accessible within Databricks, we can now explore it in preparation for modeling:

In [4]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/2015.csv,2015.csv,1806322
dbfs:/FileStore/tables/LAcasesus.csv,LAcasesus.csv,27018
dbfs:/FileStore/tables/NewyorkUSCases.csv,NewyorkUSCases.csv,24847
dbfs:/FileStore/tables/airlines.csv,airlines.csv,779982
dbfs:/FileStore/tables/coronadataset.csv,coronadataset.csv,36802504
dbfs:/FileStore/tables/coronadatasetus.csv,coronadatasetus.csv,37492352
dbfs:/FileStore/tables/coronadatasetus2.csv,coronadatasetus2.csv,26265
dbfs:/FileStore/tables/covidts.csv,covidts.csv,1169107
dbfs:/FileStore/tables/customers.csv,customers.csv,735513
dbfs:/FileStore/tables/flights.csv,flights.csv,72088113


In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

import re
from pyspark.sql.types import *

# structure of the training data set
train_schema = StructType([
  StructField('UID', IntegerType()),
  StructField('iso2', StringType()),
  StructField('iso3', StringType()),
  StructField('code3', IntegerType()),
  StructField('FIPS', IntegerType()),
  StructField('Admin2', StringType()),
  StructField('Lat', DoubleType()),
  StructField('CombinedKey', StringType()),
  StructField('Population', IntegerType()),
  StructField('Date', StringType()),
  StructField('Case', IntegerType()),
  StructField('Long', DoubleType()),
  StructField('Country', StringType()),
  StructField('Province', StringType())
  ])

# read the training file into a dataframe
#  orignal location:   '/FileStore/tables/demand_forecast/train/train.csv', 
train = spark.read.csv(
  '/FileStore/tables/usdeathny.csv', 
  header=True, 
  schema=train_schema
  )

# make the dataframe queriable as a temporary view
train.createOrReplaceTempView('train')

In [6]:
train.show()
 


+--------+----+----+-----+-----+--------+----------+--------------------+----------+----------+----+------------+-------+--------+
 UID|iso2|iso3|code3| FIPS| Admin2| Lat| CombinedKey|Population| Date|Case| Long|Country|Province|
+--------+----+----+-----+-----+--------+----------+--------------------+----------+----------+----+------------+-------+--------+
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-22| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-23| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-24| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-25| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-26| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-27| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-28| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-29| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-30| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-01-31| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-01| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-02| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-03| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-04| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-05| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-06| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-07| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-08| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-09| 0|-73.97152637| US|New York|
84036061| US| USA| 840|36061|New York|40.7672726|New York City, Ne...| 5803210|2020-02-10| 0|-73.97152637| US|New York|
+--------+----+----+-----+-----+--------+----------+--------------------+----------+----------+----+------------+-------+--------+
only showing top 20 rows

When performing demand forecasting, we are often interested in general trends and seasonality.  Let's start our exploration by examing the annual trend in unit sales:

In [8]:
%sql

SELECT
  year(Date) as year, 
  sum(Case) as case
FROM train
GROUP BY year(Date)
ORDER BY year;

year,case
2020,546064


It's very clear from the data that there is a generally upward trend in total unit sales across the stores. If we had better knowledge of the markets served by these stores, we might wish to identify whether there is a maximum growth capacity we'd expect to approach over the life of our forecast.  But without that knowledge and by just quickly eyeballing this dataset, it feels safe to assume that if our goal is to make a forecast a few days, months or even a year out, we might expect continued linear growth over that time span.

Now let's examine seasonality.  If we aggregate the data around the individual months in each year, a distinct yearly seasonal pattern is observed which seems to grow in scale with overall growth in sales:

In [10]:
%sql

SELECT 
  TRUNC(Date, 'MM') as month,
  SUM(Case) as Case
FROM train
GROUP BY TRUNC(date, 'MM')
ORDER BY month;

month,Case
2020-01-01,0
2020-02-01,0
2020-03-01,11764
2020-04-01,361925
2020-05-01,172375


Aggregating the data at a weekday level, a pronounced weekly seasonal pattern is observed with a peak on Sunday (weekday 0), a hard drop on Monday (weekday 1) and then a steady pickup over the week heading back to the Sunday high.  This pattern seems to be pretty stable across the five years of observations:

In [12]:
%sql

SELECT
  YEAR(Date) as year,
  CAST(DATE_FORMAT(Date, 'u') as Integer) % 7 as weekday,
  --CONCAT(DATE_FORMAT(date, 'u'), '-', DATE_FORMAT(date, 'EEEE')) as weekday,
  AVG(Case) as Case
FROM (
  SELECT 
    Date,
    SUM(Case) as Case
  FROM train
  GROUP BY Date
 ) x
GROUP BY year, CAST(DATE_FORMAT(Date, 'u') as Integer) --, CONCAT(DATE_FORMAT(date, 'u'), '-', DATE_FORMAT(date, 'EEEE'))
ORDER BY year, weekday;

year,weekday,Case
2020,0,4648.666666666667
2020,1,4837.6
2020,2,5008.866666666667
2020,3,4870.125
2020,4,5050.3125
2020,5,5200.375
2020,6,5419.0


Now that we are oriented to the basic patterns within our data, let's explore how we might build a forecast.

###Build a Forecast

Before attempting to generate forecasts for individual combinations of stores and items, it might be helpful to build a single forecast for no other reason than to orient ourselves to the use of FBProphet.

Our first step is to assemble the historical dataset on which we will train the model:

In [15]:
# query to aggregate data to date (ds) level
sql_statement = '''
  SELECT
    CAST(Date as date) as ds,
    Case as y
  FROM train
  WHERE Country='US' AND Admin2='New York'
  ORDER BY ds
  '''

# assemble dataset in Pandas dataframe
history_pd = spark.sql(sql_statement).toPandas()

# drop any missing records
history_pd = history_pd.dropna()

Now, we will import the fbprophet library, but because it can be a bit verbose when in use, we will need to fine-tune the logging settings in our environment:

In [17]:
from fbprophet import Prophet
import logging

# disable informational messages from fbprophet
logging.getLogger('py4j').setLevel(logging.ERROR)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.

Based on our review of the data, it looks like we should set our overall growth pattern to linear and enable the evaluation of weekly and yearly seasonal patterns. We might also wish to set our seasonality mode to multiplicative as the seasonal pattern seems to grow with overall growth in sales:

In [19]:
# set model parameters
model = Prophet(
  interval_width=0.95,
  growth='linear',
  daily_seasonality=False,
  weekly_seasonality=True,
  yearly_seasonality=True,
  seasonality_mode='multiplicative'
  )

# fit the model to historical data
model.fit(history_pd)

Out[5]: <fbprophet.forecaster.Prophet at 0x7f2885883240>

Now that we have a trained model, let's use it to build a 90-day forecast:

In [21]:
# define a dataset including both historical dates & 90-days beyond the last available date
future_pd = model.make_future_dataframe(
  periods=90, 
  freq='d', 
  include_history=True
  )

# predict over the dataset
forecast_pd = model.predict(future_pd)

display(forecast_pd)

ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
2020-01-22T00:00:00.000+0000,-3747.848879621213,-140.2431610507881,87.01410272790889,-3747.848879621213,-3747.848879621213,-0.9911820937918744,-0.9911820937918744,-0.9911820937918744,-0.003546675180436664,-0.003546675180436664,-0.003546675180436664,-0.9876354186114377,-0.9876354186114377,-0.9876354186114377,0.0,0.0,0.0,-33.04817990272852
2020-01-23T00:00:00.000+0000,-3657.9340199160833,-124.26508519934724,107.28349998720914,-3657.9340199160833,-3657.9340199160833,-0.9965749553538209,-0.9965749553538209,-0.9965749553538209,-8.364451900549279E-4,-8.364451900549279E-4,-8.364451900549279E-4,-0.9957385101637659,-0.9957385101637659,-0.9957385101637659,0.0,0.0,0.0,-12.52858733098997
2020-01-24T00:00:00.000+0000,-3568.019160210954,-64.23704984271193,170.02562152479297,-3568.019160210954,-3568.019160210954,-1.0145701731089798,-1.0145701731089798,-1.0145701731089798,-0.013189643030150648,-0.013189643030150648,-0.013189643030150648,-1.0013805300788292,-1.0013805300788292,-1.0013805300788292,0.0,0.0,0.0,51.98665682043029
2020-01-25T00:00:00.000+0000,-3478.104300505824,-118.06567454801203,112.72338870573147,-3478.104300505824,-3478.104300505824,-0.9989635611200471,-0.9989635611200471,-0.9989635611200471,0.005874959683284503,0.005874959683284503,0.005874959683284503,-1.0048385208033317,-1.0048385208033317,-1.0048385208033317,0.0,0.0,0.0,-3.6048425255754863
2020-01-26T00:00:00.000+0000,-3388.1894437782025,-130.42143428070264,109.78673775895034,-3388.1894437782025,-3388.1894437782025,-0.9981937223762906,-0.9981937223762906,-0.9981937223762906,0.008236655395127284,0.008236655395127284,0.008236655395127284,-1.0064303777714179,-1.0064303777714179,-1.0064303777714179,0.0,0.0,0.0,-6.120010777185102
2020-01-27T00:00:00.000+0000,-3298.2745870505805,-114.47187433639118,114.19145185366419,-3298.2745870505805,-3298.2745870505805,-0.9994086569569826,-0.9994086569569826,-0.9994086569569826,0.007093013856109941,0.007093013856109941,0.007093013856109941,-1.0065016708130925,-1.0065016708130925,-1.0065016708130925,0.0,0.0,0.0,-1.9504117310135967
2020-01-28T00:00:00.000+0000,-3208.3597303229585,-89.56621703989406,148.71966191287868,-3208.3597303229585,-3208.3597303229585,-1.0090437801982537,-1.0090437801982537,-1.0090437801982537,-0.00363186553389502,-0.00363186553389502,-0.00363186553389502,-1.0054119146643588,-1.0054119146643588,-1.0054119146643588,0.0,0.0,0.0,29.01570019796933
2020-01-29T00:00:00.000+0000,-3118.444873595337,-100.02394879358869,143.41977749240104,-3118.444873595337,-3118.444873595337,-1.0070674662378627,-1.0070674662378627,-1.0070674662378627,-0.003546675180472818,-0.003546675180472818,-0.003546675180472818,-1.0035207910573898,-1.0035207910573898,-1.0035207910573898,0.0,0.0,0.0,22.03950385877105
2020-01-30T00:00:00.000+0000,-3028.5300142380574,-102.01391651772349,126.9892685815513,-3028.5300142380574,-3028.5300142380574,-1.002011264101213,-1.002011264101213,-1.002011264101213,-8.364451900768123E-4,-8.364451900768123E-4,-8.364451900768123E-4,-1.0011748189111362,-1.0011748189111362,-1.0011748189111362,0.0,0.0,0.0,6.091173697083422
2020-01-31T00:00:00.000+0000,-2938.6151548807775,-78.72849661598728,152.1259351986832,-2938.6151548807775,-2938.6151548807775,-1.0118845892630859,-1.0118845892630859,-1.0118845892630859,-0.013189643030167616,-0.013189643030167616,-0.013189643030167616,-0.9986949462329183,-0.9986949462329183,-0.9986949462329183,0.0,0.0,0.0,34.92423411803756


How did our model perform? Here we can see the general and seasonal trends in our model presented as graphs:

In [23]:
trends_fig = model.plot_components(forecast_pd)
display(trends_fig)

And here, we can see how our actual and predicted data line up as well as a forecast for the future, though we will limit our graph to the last year of historical data just to keep it readable:

In [25]:
predict_fig = model.plot( forecast_pd, xlabel='Date', ylabel='Case')

# adjust figure to display dates from last year + the 90 day forecast
xlim = predict_fig.axes[0].get_xlim()
new_xlim = ( xlim[1]-(245.0), xlim[1]-45.0)
predict_fig.axes[0].set_xlim(new_xlim)

display(predict_fig)

**NOTE** This visualization is a bit busy. Bartosz Mikulski provides [an excellent breakdown](https://www.mikulskibartosz.name/prophet-plot-explained/) of it that is well worth checking out.  In a nutshell, the black dots represent our actuals with the darker blue line representing our predictions and the lighter blue band representing our (95%) uncertainty interval.

Visual inspection is useful, but a better way to evaulate the forecast is to calculate Mean Absolute Error, Mean Squared Error and Root Mean Squared Error values for the predicted relative to the actual values in our set:

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from datetime import date

# get historical actuals & predictions for comparison
actuals_pd = history_pd[ history_pd['ds'] < date(2020, 5, 6) ]['y']
predicted_pd = forecast_pd[ forecast_pd['ds'] < date(2020, 5, 6) ]['yhat']

# calculate evaluation metrics
mae = mean_absolute_error(actuals_pd, predicted_pd)
mse = mean_squared_error(actuals_pd, predicted_pd)
rmse = sqrt(mse)

# print metrics to the screen
print( '\n'.join(['MAE: {0}', 'MSE: {1}', 'RMSE: {2}']).format(mae, mse, rmse) )

/local_disk0/tmp/1589214853828-0/PythonShell.py:7: FutureWarning: Comparing Series of datetimes with 'datetime.date'. Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
 import pickle
MAE: 35.54172395431598
MSE: 3222.181103059779
RMSE: 56.76425902854523

FBProphet provides [additional means](https://facebook.github.io/prophet/docs/diagnostics.html) for evaluating how your forecasts hold up over time. You're strongly encouraged to consider using these and those additional techniques when building your forecast models but we'll skip this here to focus on the scaling challenge.

###Scaling Model Training & Forecasting

With the mechanics under our belt, let's now tackle our original goal of building numerous, fine-grain models & forecasts for individual store and item combinations.  We will start by assembling sales data at the store-item-date level of granularity:

**NOTE**: The data in this data set should already be aggregated at this level of granularity but we are explicitly aggregating to ensure we have the expected data structure.

In [31]:
sql_statement = '''
  SELECT
    Province,
    Admin2,
    CAST(Date as Date) as ds,
    SUM(Case) as y
  FROM train
  GROUP BY Province, Admin2, ds
  ORDER BY Province, Admin2, ds
  '''

store_item_history = (
  spark
    .sql( sql_statement )
    .repartition(sc.defaultParallelism, ['Province', 'Admin2'])
  ).cache()

With our data aggregated at the store-item-date level, we need to consider how we will pass our data to FBProphet. If our goal is to build a model for each store and item combination, we will need to pass in a store-item subset from the dataset we just assembled, train a model on that subset, and receive a store-item forecast back. We'd expect that forecast to be returned as a dataset with a structure like this where we retain the store and item identifiers for which the forecast was assembled and we limit the output to just the relevant subset of fields generated by the Prophet model:

In [33]:
from pyspark.sql.types import *

result_schema =StructType([
  StructField('ds',DateType()),
  StructField('Province',StringType()),
  StructField('Admin2',StringType()),
  StructField('y',FloatType()),
  StructField('yhat',FloatType()),
  StructField('yhat_upper',FloatType()),
  StructField('yhat_lower',FloatType())
  ])

To train the model and generate a forecast we will leverage a Pandas user-defined function (UDF).  We will define this function to receive a subset of data organized around a store and item combination.  It will return a forecast in the format identified in the previous cell:

In [35]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf( result_schema, PandasUDFType.GROUPED_MAP )
def forecast_store_item( history_pd ):
  
  # TRAIN MODEL AS BEFORE
  # --------------------------------------
  # remove missing values (more likely at day-store-item level)
  history_pd = history_pd.dropna()
  
  # configure the model
  model = Prophet(
    interval_width=0.95,
    growth='linear',
    daily_seasonality=False,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='multiplicative'
    )
  
  # train the model
  model.fit( history_pd )
  # --------------------------------------
  
  # BUILD FORECAST AS BEFORE
  # --------------------------------------
  # make predictions
  future_pd = model.make_future_dataframe(
    periods=90, 
    freq='d', 
    include_history=True
    )
  forecast_pd = model.predict( future_pd )  
  # --------------------------------------
  
  # ASSEMBLE EXPECTED RESULT SET
  # --------------------------------------
  # get relevant fields from forecast
  f_pd = forecast_pd[ ['ds','yhat', 'yhat_upper', 'yhat_lower'] ].set_index('ds')
  
  # get relevant fields from history
  h_pd = history_pd[['ds','Province','Admin2','y']].set_index('ds')
  
  # join history and forecast
  results_pd = f_pd.join( h_pd, how='left' )
  results_pd.reset_index(level=0, inplace=True)
  
  # get store & item from incoming data set
  results_pd['Province'] = history_pd['Province'].iloc[0]
  results_pd['Admin2'] = history_pd['Admin2'].iloc[0]
  # --------------------------------------
  
  # return expected dataset
  return results_pd[ ['ds', 'Province', 'Admin2', 'y', 'yhat', 'yhat_upper', 'yhat_lower'] ]  

There's a lot taking place within our UDF, but if you compare the first two blocks of code within which the model is being trained and a forecast is being built to the cells in the previous portion of this notebook, you'll see the code is pretty much the same as before. It's only in the assembly of the required result set that truly new code is being introduced and it consists of fairly standard Pandas dataframe manipulations.

Now let's call our UDF to build our forecasts.  We do this by grouping our historical dataset around store and item.  We then apply our UDF to each group and tack on today's date as our *training_date* for data management purposes:

In [38]:
from pyspark.sql.functions import current_date

results = (
  store_item_history
    .groupBy('Province', 'Admin2')
    .apply(forecast_store_item)
    .withColumn('training_date', current_date() )
    )

results.createOrReplaceTempView('new_forecasts5')

In [39]:
%sql
Select * from new_forecasts5;

ds,Province,Admin2,y,yhat,yhat_upper,yhat_lower,training_date
2020-01-22,New York,New York,0.0,-33.04818,85.09761,-148.37996,2020-05-11
2020-01-23,New York,New York,0.0,-12.528587,104.43173,-122.219925,2020-05-11
2020-01-24,New York,New York,0.0,51.986656,169.20236,-49.254818,2020-05-11
2020-01-25,New York,New York,0.0,-3.6048424,119.7095,-121.41416,2020-05-11
2020-01-26,New York,New York,0.0,-6.120011,113.91184,-136.90482,2020-05-11
2020-01-27,New York,New York,0.0,-1.9504117,115.08018,-115.01213,2020-05-11
2020-01-28,New York,New York,0.0,29.0157,149.48499,-79.34262,2020-05-11
2020-01-29,New York,New York,0.0,22.039503,139.41943,-103.58554,2020-05-11
2020-01-30,New York,New York,0.0,6.0911736,126.25961,-111.2929,2020-05-11
2020-01-31,New York,New York,0.0,34.924232,157.51027,-77.57351,2020-05-11


We we are likely wanting to report on our forecasts, so let's save them to a queriable table structure:

In [41]:
%sql
-- create forecast table
create table if not exists forecasts5 (
  date date,
  Province string,
  Admin2 string,
  case float,
  case_predicted float,
  case_predicted_upper float,
  case_predicted_lower float,
  training_date date
  )
using delta
partitioned by (training_date);



In [42]:
%sql
select 
  ds as date,
  Province,
  Admin2,
  y as case,
  yhat as case_predicted,
  yhat_upper as case_predicted_upper,
  yhat_lower as case_predicted_lower,
  training_date
from new_forecasts5;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,New York,New York,0.0,-33.04818,78.858406,-143.52296,2020-05-11
2020-01-23,New York,New York,0.0,-12.528587,100.704994,-127.64362,2020-05-11
2020-01-24,New York,New York,0.0,51.986656,170.18301,-61.770718,2020-05-11
2020-01-25,New York,New York,0.0,-3.6048424,122.28554,-125.82218,2020-05-11
2020-01-26,New York,New York,0.0,-6.120011,119.60825,-129.35034,2020-05-11
2020-01-27,New York,New York,0.0,-1.9504117,109.50906,-117.01991,2020-05-11
2020-01-28,New York,New York,0.0,29.0157,144.82909,-90.18693,2020-05-11
2020-01-29,New York,New York,0.0,22.039503,150.52403,-87.58541,2020-05-11
2020-01-30,New York,New York,0.0,6.0911736,126.15619,-108.457375,2020-05-11
2020-01-31,New York,New York,0.0,34.924232,143.47104,-77.939186,2020-05-11


In [43]:
 %sql
-- load data to it
insert into forecasts5
select 
  ds as date,
  Province,
  Admin2,
 y as case,
  yhat as case_predicted,
  yhat_upper as case_predicted_upper,
  yhat_lower as case_predicted_lower,
  training_date
from new_forecasts5;

In [44]:
%sql
Select * from forecasts5;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,New York,New York,0.0,-33.04818,81.88858,-147.72533,2020-05-11
2020-01-23,New York,New York,0.0,-12.528587,112.20534,-129.92749,2020-05-11
2020-01-24,New York,New York,0.0,51.986656,173.73914,-66.541756,2020-05-11
2020-01-25,New York,New York,0.0,-3.6048424,113.44343,-125.773674,2020-05-11
2020-01-26,New York,New York,0.0,-6.120011,107.4866,-134.69885,2020-05-11
2020-01-27,New York,New York,0.0,-1.9504117,113.656975,-127.571465,2020-05-11
2020-01-28,New York,New York,0.0,29.0157,146.40364,-90.32356,2020-05-11
2020-01-29,New York,New York,0.0,22.039503,140.10406,-86.478775,2020-05-11
2020-01-30,New York,New York,0.0,6.0911736,123.99876,-104.43793,2020-05-11
2020-01-31,New York,New York,0.0,34.924232,152.72086,-90.921,2020-05-11


But how good (or bad) is each forecast?  Using the UDF technique, we can generate evaluation metrics for each store-item forecast as follows:

In [46]:
import pandas as pd

# schema of expected result set
eval_schema =StructType([
  StructField('training_date', DateType()),
  StructField('Province', StringType()),
  StructField('Admin2', StringType()),
  StructField('mae', FloatType()),
  StructField('mse', FloatType()),
  StructField('rmse', FloatType())
  ])

# define udf to calculate metrics
@pandas_udf( eval_schema, PandasUDFType.GROUPED_MAP )
def evaluate_forecast( evaluation_pd ):
  
  # get store & item in incoming data set
  training_date = evaluation_pd['training_date'].iloc[0]
  Province = evaluation_pd['Province'].iloc[0]
  Admin2 = evaluation_pd['Admin2'].iloc[0]
  
  # calulate evaluation metrics
  mae = mean_absolute_error( evaluation_pd['y'], evaluation_pd['yhat'] )
  mse = mean_squared_error( evaluation_pd['y'], evaluation_pd['yhat'] )
  rmse = sqrt( mse )
  
  # assemble result set
  results = {'training_date':[training_date], 'Province':[Province], 'Admin2':[Admin2], 'mae':[mae], 'mse':[mse], 'rmse':[rmse]}
  return pd.DataFrame.from_dict( results )

# calculate metrics
results = (
  spark
    .table('new_forecasts5')
    .filter('ds < \'2020-05-06\'') # limit evaluation to periods where we have historical data
    .select('training_date', 'Province', 'Admin2', 'y', 'yhat')
    .groupBy('training_date', 'Province', 'Admin2')
    .apply(evaluate_forecast)
    )
results.createOrReplaceTempView('new_forecast_evals5')



Once again, we will likely want to report the metrics for each forecast, so we persist these to a queriable table:

In [48]:
%sql

create table if not exists forecast_evals5 (
  Province string,
  Admin2 string,
  mae float,
  mse float,
  rmse float,
  training_date date
  )
using delta
partitioned by (training_date);

insert into forecast_evals5
select
  Province,
  Admin2,
  mae,
  mse,
  rmse,
  training_date
from new_forecast_evals5;

In [49]:
%sql
Select * from forecasts5;

date,Province,Admin2,case,case_predicted,case_predicted_upper,case_predicted_lower,training_date
2020-01-22,New York,New York,0.0,-33.04818,81.88858,-147.72533,2020-05-11
2020-01-23,New York,New York,0.0,-12.528587,112.20534,-129.92749,2020-05-11
2020-01-24,New York,New York,0.0,51.986656,173.73914,-66.541756,2020-05-11
2020-01-25,New York,New York,0.0,-3.6048424,113.44343,-125.773674,2020-05-11
2020-01-26,New York,New York,0.0,-6.120011,107.4866,-134.69885,2020-05-11
2020-01-27,New York,New York,0.0,-1.9504117,113.656975,-127.571465,2020-05-11
2020-01-28,New York,New York,0.0,29.0157,146.40364,-90.32356,2020-05-11
2020-01-29,New York,New York,0.0,22.039503,140.10406,-86.478775,2020-05-11
2020-01-30,New York,New York,0.0,6.0911736,123.99876,-104.43793,2020-05-11
2020-01-31,New York,New York,0.0,34.924232,152.72086,-90.921,2020-05-11


We now have constructed a forecast for each store-item combination and generated basic evaluation metrics for each.  To see this forecast data, we can issue a simple query (limited here to product 1 across stores 1 through 10):

In [51]:
%sql

SELECT
  Province,
  date,
  Admin2,
  case_predicted,
  case_predicted_upper,
  case_predicted_lower
FROM forecasts5 a
WHERE Admin2 = 'New York' AND
      --store IN (1, 2, 3, 4, 5) AND
      date >= '2020-05-01' AND
      training_date=current_date()
ORDER BY Province

Province,date,Admin2,case_predicted,case_predicted_upper,case_predicted_lower
New York,2020-05-01,New York,18304.236,18423.895,18188.69
New York,2020-05-02,New York,18632.94,18755.064,18519.146
New York,2020-05-03,New York,18858.734,18974.314,18738.477
New York,2020-05-04,New York,19053.422,19171.797,18929.383
New York,2020-05-05,New York,19177.924,19294.69,19068.582
New York,2020-05-06,New York,19347.947,19464.51,19225.82
New York,2020-05-07,New York,19512.06,19635.713,19393.273
New York,2020-05-08,New York,19558.678,19677.38,19439.639
New York,2020-05-09,New York,19763.832,19883.97,19643.148
New York,2020-05-10,New York,19830.209,19950.889,19715.793


And for each of these, we can retrieve a measure of help us assess the reliability of each forecast:

In [53]:
%sql

SELECT
  Province,
  mae,
  mse,
  rmse
FROM forecast_evals5 a
WHERE Admin2 = 'New York' AND
      training_date=current_date()
ORDER BY Province

Province,mae,mse,rmse
New York,35.541718,3222.1748,56.764202
